## Importing Libraries



In [ ]:
import numpy as np
import pandas as pd
import re,os,time
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import gensim
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('wordnet')
nltk.download('punkt')
import random
import spacy

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Amazon Review Dataset

## collecting all Reviews for multiple csv files
\

In [ ]:
file1=pd.read_csv('/content/drive/My Drive/IRE_Data/1429_1.csv')
file2=pd.read_csv('/content/drive/My Drive/IRE_Data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv')
file3=pd.read_csv('/content/drive/My Drive/IRE_Data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')
df1=file1['reviews.text']
df2=file2['reviews.text']
df3=file3['reviews.text']

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
print(len(df1))
print(len(df2))
print(len(df3))

34660
5000
28332


### CSV file Data format

In [ ]:
print(df1.head(10))

0    This product so far has not disappointed. My c...
1    great for beginner or experienced person. Boug...
2    Inexpensive tablet for him to use and learn on...
3    I've had my Fire HD 8 two weeks now and I love...
4    I bought this for my grand daughter when she c...
5    This amazon fire 8 inch tablet is the perfect ...
6    Great for e-reading on the go, nice and light ...
7    I gave this as a Christmas gift to my inlaws, ...
8    Great as a device to read books. I like that i...
9    I love ordering books and reading them with th...
Name: reviews.text, dtype: object


In [ ]:
print(df2.tail(10))

4990    Tablets are best used for personal reading. Cl...
4991    My son uses the device to help him do his home...
4992    My 6yr old son loves the tablet!I plan on purc...
4993    I APPRECIATE the ease of purchasing and readin...
4994    I love this tablet/kindle. I can access email,...
4995    This is a great tablet for the price. Amazon i...
4996    This tablet is the perfect size and so easy to...
4997    Purchased this for my son. Has room to upgrade...
4998    I had some thoughts about getting this for a 5...
4999    this is a steal, have 8 gb model as well.This ...
Name: reviews.text, dtype: object


### concatenation of all csv files

In [ ]:
df=pd.concat([df1,df2,df3],axis=0)

In [ ]:
n_df=np.array(df)

In [ ]:
print(len(n_df))

67992


### Preprocessing of Data

In [ ]:
#https://www.kaggle.com/sudalairajkumar/getting-started-with-text-preprocessing

def remove_numbers(text): 
    result = re.sub(r'\d+', '', text) 
    return result

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

stemmer = PorterStemmer()
def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def clean_text(text):
  text=re.sub(r'["|\'|?|,)|(|-]',r'',text)
  text=remove_numbers(text)
  # text=" ".join(filter(lambda x:x[0]!='@', text.split()))
  text=remove_punctuation(text)
  text=text.lower()
  text=re.sub(r"http\S+", "",text)
  # text=lemmatize_words(text) # Time taking...removed for now
  text=stem_words(text)
  return text

preprocess= lambda x: clean_text(x)

### Reviews extracted from Amazon dataset

In [ ]:
data=[]
for line in n_df:
  data.append(clean_text(str(line)))
# clean_data=pd.DataFrame(data)

## NY Times Dataset

In [ ]:
import glob
review_files = glob.glob("/content/drive/My Drive/IRE_Data/Comments*.csv")
article_files = glob.glob("/content/drive/My Drive/IRE_Data/Articles*.csv")  

In [ ]:
df0 = pd.read_csv(article_files[0])

In [ ]:
df0.head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,By STEPHEN HILTNER and SUSAN LEHMAN,article,Finding an Expansive View of a Forgotten Peop...,"['Photography', 'New York Times', 'Niger', 'Fe...",3,Insider,2,2017-04-01 00:15:41,Unknown,One of the largest photo displays in Times his...,The New York Times,News,https://www.nytimes.com/2017/03/31/insider/nig...
1,NaN,58def3237c459f24986d7c84,823,By GAIL COLLINS,article,"And Now, the Dreaded Trump Curse","['United States Politics and Government', 'Tru...",3,OpEd,23,2017-04-01 00:23:58,Unknown,Meet the gang from under the bus.,The New York Times,Op-Ed,https://www.nytimes.com/2017/03/31/opinion/and...
2,NaN,58def9f57c459f24986d7c90,575,By THE EDITORIAL BOARD,article,Venezuela’s Descent Into Dictatorship,"['Venezuela', 'Politics and Government', 'Madu...",3,Editorial,22,2017-04-01 00:53:06,Unknown,A court ruling annulling the legislature’s aut...,The New York Times,Editorial,https://www.nytimes.com/2017/03/31/opinion/ven...
3,NaN,58defd317c459f24986d7c95,1374,By MICHAEL POWELL,article,Stain Permeates Basketball Blue Blood,"['Basketball (College)', 'University of North ...",3,Sports,1,2017-04-01 01:06:52,College Basketball,"For two decades, until 2013, North Carolina en...",The New York Times,News,https://www.nytimes.com/2017/03/31/sports/ncaa...
4,NaN,58df09b77c459f24986d7ca7,708,By DEB AMLEN,article,Taking Things for Granted,['Crossword Puzzles'],3,Games,0,2017-04-01 02:00:14,Unknown,In which Howard Barkin and Will Shortz teach u...,The New York Times,News,https://www.nytimes.com/2017/03/31/crosswords/...


In [ ]:
#Showing Complete Data Set
df0 = pd.read_csv(review_files[0])
df0.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,approveDate,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,parentID,parentUserDisplayName,permID,picURL,recommendations,recommendedFlag,replyCount,reportAbuseFlag,sharing,status,timespeople,trusted,updateDate,userDisplayName,userID,userLocation,userTitle,userURL,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,1491245186,This project makes me happy to be a 30+ year T...,22022598.0,22022598,<br/>,comment,1.491237e+09,1,False,0.0,NaN,22022598,https://graphics8.nytimes.com/images/apps/time...,2.0,NaN,0.0,NaN,0,approved,1.0,0.0,1491245186,Rob Gayle,46006296,"Riverside, CA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
1,1491188619,Stunning photos and reportage. Infuriating tha...,22017350.0,22017350,NaN,comment,1.491180e+09,1,False,0.0,NaN,22017350,https://graphics8.nytimes.com/images/apps/time...,1.0,NaN,0.0,NaN,0,approved,1.0,0.0,1491188619,Susan A.,29202761,<br/>,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
2,1491188617,Brilliant work from conception to execution. I...,22017334.0,22017334,<br/>,comment,1.491179e+09,1,False,0.0,NaN,22017334,https://graphics8.nytimes.com/images/apps/time...,3.0,NaN,0.0,NaN,0,approved,1.0,0.0,1491188617,Meta,63944806,Raleigh NC,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
3,1491167820,NYT reporters should provide a contributor's l...,22015913.0,22015913,<br/>,comment,1.491150e+09,1,False,0.0,NaN,22015913,https://graphics8.nytimes.com/images/apps/time...,7.0,NaN,2.0,NaN,0,approved,1.0,0.0,1491167820,Tom Wyrick,1266184,"Missouri, USA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
4,1491167815,Could only have been done in print. Stunning.,22015466.0,22015466,<br/>,comment,1.491147e+09,1,False,0.0,NaN,22015466,http://profile.ak.fbcdn.net/hprofile-ak-snc4/h...,5.0,NaN,0.0,NaN,0,approved,1.0,0.0,1491167815,Joe Sharkey,61121360,"Tucson, Arizona",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News


###Collecting reviews from NY datasets

In [ ]:
ny_comment_df = pd.DataFrame()
tmp = []
for i in range(len(review_files)):
  temp_df = pd.DataFrame()
  temp_df = pd.read_csv(review_files[i])
  tmp.append(temp_df['commentBody'])
ny_comment_df = pd.concat(tmp)  

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,15,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/li

In [ ]:
ny_comment_df = pd.DataFrame(ny_comment_df)

In [ ]:
ny_comment_df

,commentBody
0,This project makes me happy to be a 30+ year T...
1,Stunning photos and reportage. Infuriating tha...
2,Brilliant work from conception to execution. I...
3,NYT reporters should provide a contributor's l...
4,Could only have been done in print. Stunning.
...,...
264919,This year was uniquely unoriginal - the four n...
264920,"Come From Away, last year..."
264921,I love this quiet little musical. Exquisite pe...
264922,We saw it about 10 days ago. Agree with your ...


## Taking 50000 Reviews from NY Dataset

In [ ]:
# taking for 50000 comments only for now
x = ny_comment_df['commentBody'][:50000].apply(clean_text)

In [ ]:
x

0        thi project make me happi to be a year time su...
1        stun photo and reportag infuri that the trump ...
2        brilliant work from concept to execut ive neve...
3        nyt report should provid a contributor link to...
4                  could onli have been done in print stun
                               ...                        
49995    bill clinton i forgot it okay to attack the ac...
49996    what about what bill clinton jfk lyndon johnso...
49997    so what new just like bill clinton jfk rfk lyn...
49998    fox the pimp for old unattract white men invol...
49999    they should all be lock upbrthrow away the key...
Name: commentBody, Length: 50000, dtype: object

## Concatenation of reviews from NY Dataset and Amazon Dataset

In [ ]:
for line in x:
  data.append(line)

## Generating templates from reviews

In [ ]:
import random

def mask_data(text, mask, strategy="random"):
    """
    strategy: 
        1. begining: masks words at the begining
        2. ending: masks words towards the end
        3. random: masks words at random positions.
    """

    stext = text.split()
    n = len(stext)
    if n>2:
      m = random.randint(1, n//2)
    else:
      return ""
    if strategy == "begining":
        tmp= [mask]
        tmp.extend(stext[m:])
        stext = tmp
    elif strategy == "ending":
        stext = stext[:n-m]
        stext.append(mask)   
    elif strategy == "random":
        k = random.randrange(n)
        # stext[k:3] = mask
        tmp=[]
        tmp.extend(stext[:k])
        tmp.append(mask)
        if n-k-1>1:
          c = random.randrange(n-k-1)
        else:
          c=3
        tmp.extend(stext[c+k:])
        stext=tmp
    else:
        print("Ill format strategy")
    return " ".join(stext)


In [ ]:
options=["random","begining","ending","random","random","random"]
flag=None
masked_text_X=[]
masked_text_Y=[]
count=0
for line in data:
  count+=1
  flag=random.randint(0, 5)
  li=mask_data(line,"***mask***",options[flag])
  if len(li)<2:
    continue
  masked_text_X.append(li)
  masked_text_Y.append(line)  

#sample output
print("total masked templates : ",len(masked_text_X))
print("masked input :",masked_text_X[120])
print("masked output :",masked_text_Y[120])

total masked templates :  115443
masked input : a great christma gift the ***mask*** love the gift
masked output : a great christma gift the kid love the gift


In [ ]:
ny_article_df = pd.DataFrame()
tmp = []
for i in range(len(article_files)):
  df = pd.DataFrame()
  summary = []
  tdf = pd.read_csv(article_files[i])
  df['headline'] = tdf['headline']
  df['keywords'] = tdf['keywords']
  df['URL'] = tdf['webURL']
  tmp.append(df)
ny_article_df = pd.concat(tmp)  

### Text Ranking for extracting Important keywords

In [ ]:
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests


In [ ]:
# !pip install lxml
!pip install pytextrank
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
def extract_keywords_(url):
  page=urlopen(url)
  soup=BeautifulSoup(page,"html.parser")
  text=' '.join(map(lambda p:p.text, soup.find_all('p')))
  summary=Textsummary(text)
  Tokens=Tokenize(summary)
  return Tokens
  
def extract_summary_(url):
  page=urlopen(url)
  soup=BeautifulSoup(page,"html.parser")
  text=' '.join(map(lambda p:p.text, soup.find_all('p')))
  summary=Textsummary(text)
  return summary

def Textsummary(text):
  summary=clean_text(summarize(text,word_count=100))
  return summary

def Tokenize(summary):
  nlp = spacy.load('en_core_web_sm')
  doc = nlp(summary)
  candidate_pos = ['NOUN', 'PROPN', 'VERB']
  sentences = []
  for sent in doc.sents:
    selected_words = []
    # print(sent)
    for token in sent:
        if token.pos_ in candidate_pos and token.is_stop is False:
            sentences.append(token)
  return sentences

#### sample URL given to retrieve important keywords from document

In [ ]:
#text from url
url="https://en.wikipedia.org/wiki/Article_370_of_the_Constitution_of_India"
keywords=extract_keywords_(url)
print(keywords)

[effect, presidenti, order, issu, sinc, extend, subject, union, list, power, govern, state, jammu, kashmir, articl, constitut, india, order, amend, order, replac, presum, becaus, constitution, wa, doubt, accord, cottrel]


### Articles and their important keywords

In [ ]:
def listToString(s):  
    str1 = ""  
    bad_chars = [';', ':','[',']','!',"'","*"]
    for i in bad_chars:
      s = s.replace(i,'')
    return s.split(',')
g = ny_article_df['keywords'].to_numpy()
li = []
for i in range(len(g)):
  li.append(listToString(g[i]))
ny_article_df['summary']=li
print(li)  


[['Photography', ' New York Times', ' Niger', ' Ferguson', ' Adam (1978- )', ' Boko Haram'], ['United States Politics and Government', ' Trump', ' Donald J', ' Spicer', ' Sean M (1971- )', ' Ryan', ' Paul D Jr', ' Priebus', ' Reince R', ' Nunes', ' Devin G', ' Christie', ' Christopher J', ' Flynn', ' Michael G (1983- )', ' Mines and Mining', ' Russia'], ['Venezuela', ' Politics and Government', ' Maduro', ' Nicolas'], ['Basketball (College)', ' University of North Carolina', ' Williams', ' Roy A', ' Cheating', ' NCAA Basketball Championships (Men)'], ['Crossword Puzzles'], ['Theater', ' Park Avenue Armory (Manhattan', ' NY)', ' The Hairy Ape (Play)', ' "ONeill', ' Eugene"', ' Cannavale', ' Bobby', ' Jones', ' Richard (1953- )', ' Laing', ' Stewart'], ['Travel and Vacations', ' Vancouver (British Columbia)'], ['"OReilly', ' Bill"', ' Fox News Channel', ' Sexual Harassment', ' 21st Century Fox', ' News and News Media'], ['Mice', ' Real Estate and Housing (Residential)'], ['Taxation', ' T

In [ ]:
ny_article_df = ny_article_df.drop(['keywords'], axis = 1) 
print(ny_article_df)

                                              headline  ...                                            summary
0    Finding an Expansive View  of a Forgotten Peop...  ...  [Photography,  New York Times,  Niger,  Fergus...
1                    And Now,  the Dreaded Trump Curse  ...  [United States Politics and Government,  Trump...
2                Venezuela’s Descent Into Dictatorship  ...  [Venezuela,  Politics and Government,  Maduro,...
3                Stain Permeates Basketball Blue Blood  ...  [Basketball (College),  University of North Ca...
4                            Taking Things for Granted  ...                                [Crossword Puzzles]
..                                                 ...  ...                                                ...
880                        A Different Bargain on Race  ...  [Reparations,  Blacks,  Affirmative Action,  M...
881        She’s 17 and Needs Birth Control. What Now?  ...  [Health Insurance and Managed Care,  Birth Con...
8

In [ ]:
import random
templates=masked_text_X
random.shuffle(templates)

In [ ]:
Titles, keywords_list_, URLs = ny_article_df['headline'],ny_article_df['summary'],ny_article_df['URL']

In [ ]:
print (URLs)

0      https://www.nytimes.com/2017/03/31/insider/nig...
1      https://www.nytimes.com/2017/03/31/opinion/and...
2      https://www.nytimes.com/2017/03/31/opinion/ven...
3      https://www.nytimes.com/2017/03/31/sports/ncaa...
4      https://www.nytimes.com/2017/03/31/crosswords/...
                             ...                        
880    https://www.nytimes.com/2017/03/04/opinion/sun...
881    https://www.nytimes.com/2017/03/04/opinion/sun...
882    https://www.nytimes.com/2017/03/04/opinion/sun...
883    https://www.nytimes.com/2017/03/04/opinion/sun...
884    https://www.nytimes.com/2017/03/04/public-edit...
Name: URL, Length: 9335, dtype: object


##FitBert Model

### FitBert Model is used to fill **single** mask sentence

In [ ]:
!pip install fitbert
from fitbert import FitBert

     |████████████████████████████████| 225kB 5.5MB/s 
     |████████████████████████████████| 1.3MB 7.1MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 1.1MB 15.6MB/s 
     |████████████████████████████████| 890kB 20.7MB/s 
     |████████████████████████████████| 2.9MB 29.4MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
  Created wheel for fitbert: filename=fitbert-0.9.0-cp36-none-any.whl size=214163 sha256=e3adc245b6a9dab0b141e1df6dc86f5df50c7531760ea95a173b89078b7e7846
  Stored in directory: /root/.cache/pip/wheels/96/e2/5b/b847ce6a5fac79014a603441cc446fda9e6f7099da0da8d8b0
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=612a360080fc3ad6234c5325f2c44bc68c34cfe6c8ec59f6a997c92b558908ff
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
  Created wheel for dill: filename=dill-0.2.5-cp36-none-any.whl size=52423 sha256=8da

### Predicting Reviews for each Document

In [44]:
def predict_reviews_(templates,keywords):
  templates_set=random.sample(templates,5)
  # numberOfTemplates=len(templates_set)
  # predicted_reviews=[]
  # for EachTemplate in templates_set:
  #   rev=fb.fitb(EachTemplate,options=keywords,delemmatize=True)
  #   predicted_reviews.append(rev)
  # return predicted_reviews
  return templates_set

#### Model FitBert Initialization

In [ ]:
fb = FitBert()

device: cpu
using model: bert-large-uncased


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Loading GloVe pre-trained Model

In [ ]:
def loadGloveModel(glove_file):
    f = codecs.open(glove_file + ".txt", 'r', encoding='utf-8')
    model = {}
    for line in f[1:]:
        split_line = line.split()
        word = split_line[0]
        embedding = np.array([float(val) for val in split_line[1:]])
        model[word] = embedding
    return model

model = loadGloveModel(gloveFile)

In [ ]:
def Semantic_similarity_(s1, s2):
    import scipy
    vector_1 = np.mean([model[word] for word in preprocess(s1)],axis=0)
    vector_2 = np.mean([model[word] for word in preprocess(s2)],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    return 1-cosine

#### For each articles set of reviews are generated

In [48]:
count=0
for (options,title,url) in zip(keywords_list_,Titles,URLs):
  count+=1
  generated_reviews=predict_reviews_(templates,options)
  print("Article Title :",title)
  print("generated reviews are :",generated_reviews)
  NumberOfReviews=len(generated_reviews)
  index=0
  maxscore=0
  Article_summary=extract_summary_(url)
  for i in range(NumberOfReviews):
    currscore=semantic_similarity_(Article_summary,generated_reviews[i])
    if maxscore<currscore:
      index=i
      maxscore=currscore
  print("Title Of Document : ",title)
  print("Article URL : ",url)
  print("Options : ",options)
  print("predicted-review : ",generated_reviews[index])
  if count>1:
    break
  

Title Of Document : Finding an Expansive View  of a Forgotten People in Niger
Article URL : https://www.nytimes.com/2017/03/31/insider/niger-migrants-boko-haram-photography.html
Options: ['Photography', ' New York Times', ' Niger', ' Ferguson', ' Adam (1978- )', ' Boko Haram']
Template: ***mask*** will haunt us for mani year if mike penc is to take over we are not a whole lot better off he is danger regress and in the orbit of trump a complet milquetoast penc serv as presid a joke and an insult i see him leav anyon in the administr that isnt jail remov along with the donald would the time pleas stay on penc he need to go along with trump
predicted-review: Boko Haram will haunt us for mani year if mike penc is to take over we are not a whole lot better off he is danger regress and in the orbit of trump a complet milquetoast penc serv as presid a joke and an insult i see him leav anyon in the administr that isnt jail remov along with the donald would the time pleas stay on penc he need t

###Roberta Model is used to fill **multiple** mask in the sentence

In [ ]:
!pip install transformers==2.10.0
!pip install RobertaTokenizer 
!pip install RobertaForMaskedLM
!pip install torch

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

# sentence = "Tom has fully ___ ___ ___ illness."
# I’m designing a document and don’t want to get bogged down in what the text actually says.
sentence = "I’m designing a document and don’t want to get bogged down in what the ___ ___ ___."
sentence = "My children to use it and I like the ___ ___ ___ control what content they see with ease."


def get_prediction (sent):
    
    token_ids = tokenizer.encode(sent, return_tensors='pt')
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position ]

    with torch.no_grad():
        output = model(token_ids)

    last_hidden_state = output[0].squeeze()

    list_of_list =[]
    for index,mask_index in enumerate(masked_pos):
        mask_hidden_state = last_hidden_state[mask_index]
        idx = torch.topk(mask_hidden_state, k=5, dim=0)[1]
        words = [tokenizer.decode(i.item()).strip() for i in idx]
        list_of_list.append(words)
        print ("Mask ",index+1,"Guesses : ",words)
    
    best_guess = ""
    for j in list_of_list:
        best_guess = best_guess+" "+j[0]
        
    return best_guess


print ("Original Sentence: ",sentence)
sentence = sentence.replace("___","<mask>")
print ("Original Sentence replaced with mask: ",sentence)
print ("\n")

predicted_blanks = get_prediction(sentence)
print ("\nBest guess for fill in the blank :::",predicted_blanks)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Original Sentence:  My children to use it and I like the ___ ___ ___ control what content they see with ease.
Original Sentence replaced with mask:  My children to use it and I like the <mask> <mask> <mask> control what content they see with ease.


Mask  1 Guesses :  ['fact', 'way', 'idea', 'ability', 'flexibility']
Mask  2 Guesses :  ['they', 'I', 'that', 'we', 'to']
Mask  3 Guesses :  ['can', 'they', 'I', 'could', 'to']

Best guess for fill in the blank :::  fact they can
